# GSPE Example File
This notebook runs HOPP and applies the correct power electronics efficency factors established in the Green Steel Power Electronics (GSPE) simulator

NOTE: all of the economic analysis will use the efficency factors that are default in PySAM/set in HOPP. The goal of this script is to solely return a power time series with teh correct efficency factors.

In [2]:
from hopp.simulation import HoppInterface
import numpy as np

hi = HoppInterface("./inputs/08-GSPE.yaml")
hi.simulate(30)



print("simulation complete")

simulation complete


## Work on implementing new power electronic efficencies according to the GSPE code

1) revert efficency factors applied in HOPP

In [ ]:
gen, annual_energy, capacity_factor = dict()

with hi.hopp.system.technologies['wind']._system_model as wp:
    if wp.model_name == 'floris':
        DEFAULT_PYSAM_LOSSES = 12.83
        gen['wind'] = wp.gen * ((100 - DEFAULT_PYSAM_LOSSES)/100) ** -1 # revert PySAM default losses
        annual_energy['wind'] = np.sum(wp.gen)
        capacity_factor['wind'] = np.sum(wp.gen) / (8760 * wp.system_capacity) * 100

    else: # wp.config.model_name == 'pysam'
        non_wake_losses = 0
        pysam_losses_dict = wp.export()
        relevant_losses = ['avail_bop_loss', 'avail_grid_loss', 'avail_turb_loss', 'elec_eff_loss', 'elec_parasitic_loss', 
                           'env_degrad_loss', 'env_env_loss', 'env_exposure_loss', 'env_icing_loss', 'ops_env_loss', 
                           'ops_grid_loss', 'ops_load_loss', 'ops_strategies_loss']
        
        for losses in relevant_losses:
            non_wake_losses += pysam_losses_dict[losses]
        
        gen['wind'] = wp.system_model.gen * ((100 - non_wake_losses)/100) ** -1 
        annual_energy['wind'] = np.sum(wp.gen)
        capacity_factor['wind'] = np.sum(wp.gen) / (8760 * wp.system_capacity) * 100

with hi.hopp.system.technologies['pv']._system_model as pv:
    pysam_losses = ((100 - pv.SystemDesign.inv_eff) + pv.SystemDesign.losses) / 100

    gen['pv'] = pv.ac * pysam_losses ** -1
    annual_energy['pv'] = np.sum(gen['pv'])
    capacity_factor['pv'] = np.sum(gen['pv']) / (8760 * pv.SystemDesign.system_capacity) * 100
    
with hi.hopp.system.technologies['battery'] as battery:
    gen['battery'] = battery.generation_profile
    annual_energy['battery'] = np.sum(gen['battery'])
    capacity_factor['battery'] = np.sum(gen['battery']) / (8760 * battery.system_capacity_kw) * 100



2) calculate losses according to GSPE code

In [ ]:
eff = {'back_to_back': 0.9604,
       'diode_based_rectifier': 0.96,
       'thyristor_based_rectifier': 0.992,
       'IGBT_based_inverter': 0.98,
       'bi_DCDC_converter': 0.985,
       'uni_DCDC_converter': 0.985,
       'transformer_up': 0.99,
       'transformer_down': 0.99,
       'rectifier_power_factor': 0.89,
       'transformer_power_factor': 0.85,
       }

# TODO instead of "gutting" the code, could package the file and then use as module
DFIG_AC_1_eff_wind = eff['transformer_up'] * eff['transformer_down'] * eff['diode_based_rectifier'] * 100 # TODO excluding battery losses here... is that what I am supposed to do?
DFIG_AC_1_eff_solar = eff['IGBT_based_inverter'] * eff['transformer_down'] * eff['diode_based_rectifier'] * 100 
DFIG_AC_1_eff_battery = eff['bi_DCDC_converter'] * 100 # assuming bi-directional... is this right?

efficencies = {'wind': DFIG_AC_1_eff_wind, 'pv': DFIG_AC_1_eff_solar, 'battery': DFIG_AC_1_eff_battery}

# apply each of the efficencies
for d in [gen, annual_energy, capacity_factor]:
       for k in d.keys(): 
              d[k] = d[k] * efficencies[k]



# TODO apply the above efficency to the powers in the HI
# TODO this is done after the hopp simulaiton is complete, meaning any 
# costs assoicated with the power generation are going to have to be recalculated...
# not sure if there is a better way of doing this without completely changing HOPP

